In [10]:
import numpy as np
import pandas as pd
from math import isnan
from datetime import datetime,date
# Saved Tiingo API Token in 'token' variable in apiToken.py file
from apiToken import token

In [11]:
# Getting prices from Tiingo
def getPrice(token,ticker,startDate,endDate):
    return pd.read_csv('https://api.tiingo.com/tiingo/daily/'+ticker+'/prices?startDate='+startDate+'&endDate='+endDate+'&token='+token+'&format=csv')
today = str(date.today())
bndPrice = getPrice(token,'bnd','2019-09-30',today)
bndxPrice = getPrice(token,'bndx','2019-09-30',today)
vtiPrice = getPrice(token,'vti','2019-09-30',today)
vxusPrice = getPrice(token,'vxus','2019-09-30',today)
vtiPrice['returns'] = (vtiPrice.adjClose)/vtiPrice.adjClose.shift(1) - 1
vxusPrice['returns'] = (vxusPrice.adjClose)/vxusPrice.adjClose.shift(1) - 1
bndPrice['returns'] = (bndPrice.adjClose)/bndPrice.adjClose.shift(1) - 1
bndxPrice['returns'] = (bndxPrice.adjClose)/bndxPrice.adjClose.shift(1) - 1
bndPrice = bndPrice.loc[:,['date','returns','adjClose']]
bndxPrice = bndxPrice.loc[:,['date','returns','adjClose']]
vtiPrice = vtiPrice.loc[:,['date','returns','adjClose']]
vxusPrice = vxusPrice.loc[:,['date','returns','adjClose']]

In [14]:
import os
fileName = input("Please enter the name of the file you'd like to use.")
while not os.path.isfile(fileName):
    fileName = input("Whoops! No such file! Please enter the name of the file you'd like to use.")

In [17]:
# Reading saved data (Market cap is in billions)
vti = pd.read_csv('Data/vti_MC_data.csv')
vxus = pd.read_csv('Data/vxus_MC_data.csv')
bnd = pd.read_csv('Data/bnd_MC_data.csv')
bndx = pd.read_csv('Data/bndx_MC_data.csv')

In [18]:
df = vtiPrice.rename(columns={"returns":"VTI_Return","adjClose":"VTI_Price"}).merge(vxusPrice.rename(columns={"returns":"VXUS_Return","adjClose":"VXUS_Price"}),on='date',how='outer').merge(bndPrice.rename(columns={"returns":"BND_Return","adjClose":"BND_Price"}),on='date',how='outer').merge(bndxPrice.rename(columns={"returns":"BNDX_Return","adjClose":"BNDX_Price"}),on='date',how='outer').merge(vti.rename(columns={"Market Cap":"VTI_MarketCap"}),on='date',how='left').merge(vxus.rename(columns={"Market Cap":"VXUS_MarketCap"}),on='date',how='left').merge(bnd.rename(columns={"Market Cap":"BND_MarketCap"}),on='date',how='left').merge(bndx.rename(columns={"Market Cap":"BNDX_MarketCap"}),on='date',how='left').sort_values(by='date').reset_index(drop=True)

In [19]:
etfs = ['VTI','VXUS','BND','BNDX']
for etf in etfs:
    list = []
    i = 0
    value = 999
    for mc,ret in zip(df[etf+'_MarketCap'],df[etf+'_Return']):
        if i>0:
            value = (ret+1) * list[i-1]
        list.append(mc) if not isnan(mc) else list.append(value)
        i+=1
    df[etf+'_MarketCap'] = list

In [20]:
df['VTI_weight'] = df['VTI_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['VXUS_weight'] = df['VXUS_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['BND_weight'] = df['BND_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)
df['BNDX_weight'] = df['BNDX_MarketCap']/(df.VTI_MarketCap+df.VXUS_MarketCap+df.BND_MarketCap+df.BNDX_MarketCap)

#Enter starting amount
startingAmount = 10000

portfolioVal = [0 for i in range(len(df))]
portfolioVal[0] = startingAmount
df['Portfolio_Value'] = portfolioVal

In [21]:
i=0
list2 = []
portfolioRet = 1
for vtiR,vxusR,bndR,bndxR,vtiW,vxusW,bndW,bndxW,portfolioValue in zip(df.VTI_Return,df.VXUS_Return,df.BND_Return,df.BNDX_Return,df.VTI_weight.shift(1),df.VXUS_weight.shift(1),df.BND_weight.shift(1),df.BNDX_weight.shift(1),df.Portfolio_Value):
    if i>0:
        portfolioRet = (vtiR*vtiW+vxusR*vxusW+bndR*bndW+bndxR*bndxW)+1
    list2.append(portfolioValue) if portfolioValue != 0 else list2.append(list2[i-1]*portfolioRet)
    i+=1
df['Portfolio_Value'] = list2

In [22]:
df

,date,VTI_Return,VTI_Price,VXUS_Return,VXUS_Price,BND_Return,BND_Price,BNDX_Return,BNDX_Price,VTI_MarketCap,VXUS_MarketCap,BND_MarketCap,BNDX_MarketCap,VTI_weight,VXUS_weight,BND_weight,BNDX_weight,Portfolio_Value
0,2019-09-30,NaN,148.218599,NaN,50.350477,NaN,82.118147,NaN,56.844145,29401.430000,999.000000,22512.530000,19421.010000,0.406468,0.013811,0.311230,0.268491,10000.000000
1,2019-10-01,-0.013046,146.284886,-0.009098,49.892391,0.000967,82.197567,-0.001668,56.749356,29017.848430,989.911150,22534.302867,19388.625144,0.403414,0.013762,0.313278,0.269546,9944.247162
2,2019-10-02,-0.016238,143.909462,-0.015628,49.112670,0.002253,82.382763,0.000000,56.749356,28546.646704,974.440767,22585.073856,19388.625144,0.399283,0.013630,0.315898,0.271189,9883.984865
3,2019-10-03,0.008117,145.077543,0.007740,49.492784,0.003313,82.655682,0.003749,56.962118,28778.353338,981.982578,22659.894262,19461.315968,0.400358,0.013661,0.315239,0.270741,9937.453474
4,2019-10-04,0.013058,146.971992,0.006696,49.824166,0.001651,82.792142,0.000170,56.971789,29154.146450,988.557491,22697.304465,19464.620096,0.403213,0.013672,0.313912,0.269203,9995.943607
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,2020-11-03,0.019202,171.970000,0.019350,52.680000,-0.000228,87.530000,-0.000686,58.310000,33615.426550,24852.992138,23611.818021,22224.864550,0.322280,0.238272,0.226373,0.213076,10765.970731
278,2020-11-04,0.021806,175.720000,0.015566,53.500000,0.006855,88.130000,0.002401,58.450000,34348.448877,25239.845850,23773.672138,22278.225570,0.325146,0.238923,0.225044,0.210888,10903.773699
279,2020-11-05,0.020658,179.350000,0.021682,54.660000,0.001021,88.220000,0.000171,58.460000,35058.014489,25787.102321,23797.950255,22282.037071,0.327874,0.241170,0.222567,0.208389,11036.397335
280,2020-11-06,-0.000502,179.260000,0.003110,54.830000,-0.002267,88.020000,-0.000684,58.420000,35040.421954,25867.303700,23743.998883,22266.791066,0.327730,0.241935,0.222076,0.208259,11035.717292
